# Build a Crew to Tailor Job Applications

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from utils import get_openai_api_key, get_serper_api_key

os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [ ]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='W:\CrewAI\L2\Vansh_Dugar_Resume.md')
semantic_search_resume = MDXSearchTool(mdx='W:\CrewAI\L2\Vansh_Dugar_Resume.md')

## Creating Agents

In [5]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [6]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [7]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [8]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [9]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
)

In [10]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [11]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [12]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [13]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=False
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
job_application_inputs = {
    'job_posting_url': 'https://www.rampp.ai/jobs/lead-data-scientist',
    'github_url': 'https://github.com/Vanshdugar',
    'personal_writeup': """Vansh Dugar is an Integrated M.Tech student in Artificial Intelligence at VIT Bhopal, with a strong interest in building impactful AI solutions. He has hands-on experience with large language models, computer vision, and multi-agent systems, and has worked on real-world applications like autonomous content generation, facial recognition for social good, and AI-powered chatbots.
    At Xoxoday, he contributed to developing AI assistants that significantly improved customer and HR operations. Vansh is passionate about blending technical innovation with user-focused product thinking to build scalable, intelligent systems."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://www.rampp.ai/jobs/lead-data-scientist) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.


# Agent: Tech Job Researcher
## Thought: Thought: I need to read the content of the job posting at the provided URL to capture all the requirements accurately.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.rampp.ai/jobs/lead-data-scientist\"}"
## Tool Output: 
Hiring Lead Data Scientist at rampp.ai Lead Data Scientist The Lead Data Scientist is a detail oriented forward-thinking individual who will utilize data mining, data analysis, machine learning and natural language processing to bring innovation and differentiated capabilities to RADI Intelligence platform. You will be at ease with contemporary machine learning, natural language processing frameworks and quantitative approaches 

w:\CrewAI\L2\venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x




# Agent: Resume Strategist for Engineers
## Thought: Thought: I will first search the MDX content for any additional relevant details about Vansh Dugar's resume specifically related to skills, projects, and experiences that match the job requirements for the Lead Data Scientist position.
## Using tool: Search a MDX's content
## Tool Input: 
"{\"search_query\": \"Lead Data Scientist relevant skills and projects Vansh Dugar\"}"
## Tool Output: 
Relevant Content:
# Vansh Dugar  
📞 +91 9385526636 | 📧 [vansh.d.work@gmail.com](mailto:vansh.d.work@gmail.com)  
🔗 [LinkedIn](https://linkedin.com/in/vansh-dugar) | 💻 [GitHub](https://github.com/Vanshdugar)

---

## 🎓 Education

**Vellore Institute of Technology (VIT), Bhopal, IN**  
_Integrated Master of Technology in Artificial Intelligence_  
**CGPA**: 8.72  
**Duration**: July 2021 – May 2026

**Nandha Central City School, Erode, IN**  
_Class 12 (CBSE)_: 93.8% (May 2021)  
_Class 10 (CBSE)_: 88% (May 2019)

---

## 🛠 Projects

### **Multi-A

- Dislplay the generated `tailored_resume.md` file.

In [21]:
from IPython.display import Markdown, display

display(Markdown("./tailored_resume.md"))

```
# Vansh Dugar  
📞 +91 9385526636 | 📧 vansh.d.work@gmail.com  
🔗 LinkedIn: https://linkedin.com/in/vansh-dugar | 💻 GitHub: https://github.com/Vanshdugar  
📍 Bangalore, India

---

## Professional Summary

Innovative and detail-oriented Data Scientist specializing in Artificial Intelligence with over 3 years of practical experience (academic and project-based) in machine learning, natural language processing (NLP), and deep learning. Proven ability to lead end-to-end AI/ML projects from problem identification to solution deployment, collaborating closely with stakeholders to translate business objectives into technical solutions. Skilled in advanced ML algorithms, state-of-the-art NLP techniques, and cloud computing platforms. Experienced in building scalable AI systems including multi-agent architectures, large language model fine-tuning, and computer vision solutions for social impact. Strong communicator adept at presenting complex technical concepts to diverse audiences and facilitating cross-functional collaboration. Enthusiastic about leveraging generative AI and AI governance principles to deliver innovative, ethically sound intelligent products.

---

## Education

**Integrated Master of Technology in Artificial Intelligence**  
Vellore Institute of Technology (VIT), Bhopal, India  
July 2021 – May 2026 | CGPA: 8.72

**Senior Secondary Education (CBSE)**  
Nandha Central City School, Erode, India  
Class 12: 93.8% (May 2021) | Class 10: 88% (May 2019)

---

## Technical Skills

- **Programming Languages & Tools:** Python, Jupyter Notebook, Java, JavaScript
- **Machine Learning & AI Frameworks:** scikit-learn, TensorFlow, PyTorch, OpenAI API, LangChain, CrewAI
- **ML Techniques:** Supervised and unsupervised learning; k-NN, Naive Bayes, SVM, Decision Forests, Logistic Regression, Multilayer Perceptrons (MLPs), Recurrent Neural Networks (RNNs), CNNs (EfficientNet)
- **Natural Language Processing:** Text mining, parsing, classification, information retrieval, NLP, Natural Language Understanding (NLU), Neural Language Modeling, NLTK, Gensim
- **Generative AI:** Large language model fine-tuning (Meta’s LLaMA), generative AI workflows
- **Data Manipulation & Analysis:** Pandas, NumPy, SciPy
- **Cloud & Databases:** Experience with cloud computing solutions and relational databases (hands-on with cloud platforms and real-time data pipelines)
- **Other:** Real-time web scraping/search integration, image processing, Aadhaar data handling, AI governance

---

## Professional Experience & Projects

### Multi-Agent Article Generator (May 2025)  
- Led the design and implementation of a multi-agent AI system using CrewAI that autonomously generates articles by simulating distinct roles such as Researcher, Writer, and Editor.  
- Integrated real-time internet search capabilities with LangChain to enhance the contextual and factual accuracy of generated content.  
- Developed agent workflows emphasizing tool and memory management to optimize language model orchestration and function execution, demonstrating end-to-end project ownership and technical innovation.

### Viskk – LLM Fine-tuning and Automated Email Handling (Oct 2024)  
- Engineered AI-powered tools to optimize email workflow, including automatic reply generation and compositional assistance, improving operational efficiency.  
- Built neural mail composition system leveraging fine-tuning of Meta’s LLaMA, demonstrating expertise in large-scale LLM customization for business applications.

### Khoya Paya – Facial Recognition System for Social Good (May 2024)  
- Developed a computer vision–based facial recognition application integrated with government Aadhaar databases to aid authorities in locating missing persons.  
- Applied advanced image processing and deep learning CNN techniques (EfficientNet), showcasing ability to implement socially impactful AI solutions.

### CNN-Based Fruit Quality Detection (Aug 2023)  
- Designed and implemented a convolutional neural network model based on EfficientNet for assessing fruit quality to address agricultural supply chain challenges.  

### AI Assistant Development at Xoxoday  
- Contributed to the development of AI assistants improving customer service and human resource operations by leveraging NLP and machine learning, collaborating actively with founders and business teams to align AI solutions with organizational goals.

---

## Communication & Leadership

- Strong collaboration skills with founders, business stakeholders, and technical teams to define project roadmaps and deliver impactful AI solutions.  
- Skilled in communicating complex technical concepts and results clearly to both technical and non-technical audiences.  
- Committed to continuous learning, ethical AI development, and driving innovation in AI-powered products.

---

## Summary

A motivated AI specialist with advanced academic training and hands-on experience in NLP, machine learning, and AI product innovation. Demonstrated ability to lead complex AI projects, deliver scalable and impactful solutions, and collaborate effectively across technical and business domains. Poised to contribute as a Lead Data Scientist by leveraging expertise in NLP, ML, generative AI, cloud platforms, and stakeholder engagement to accelerate RADI Intelligence platform growth at Rampp.ai.
```

- Dislplay the generated `interview_materials.md` file.

In [17]:
display(Markdown("./interview_materials.md"))

---
## Potential Interview Questions and Talking Points for Vansh Dugar — Lead Data Scientist Role at Rampp.ai

### 1. Experience and Background
**Question:**  
- Can you walk us through your experience with end-to-end project ownership in AI/ML, particularly from recognizing business problems to deploying solutions?  
- How have you collaborated with founders or business stakeholders to translate business objectives into technical project roadmaps?  

**Talking Points:**  
- Led multiple projects such as the Multi-Agent Article Generator showcasing full lifecycle ownership from conceptualization to deployment.  
- At Xoxoday, worked closely with founders and business teams to align AI assistant development with organizational goals, illustrating stakeholder collaboration and business-technology translation skills.

### 2. Machine Learning & NLP Expertise
**Question:**  
- What machine learning algorithms and techniques have you used most extensively? Can you give examples involving supervised and unsupervised learning?  
- Describe your experience with NLP and text analytics, including any work with text mining, classification, or information retrieval.  

**Talking Points:**  
- Applied advanced ML algorithms including k-NN, Naive Bayes, SVM, Decision Forests, Logistic Regression, MLPs, and RNNs through academic projects and fine-tuning LLMs such as Meta’s LLaMA.  
- Developed NLP systems for email automation (Viskk project) and multi-agent autonomous article generation with real-time internet search integration, demonstrating robust text mining, parsing, classification, and retrieval abilities.  
- Knowledge of key NLP libraries (NLTK, Gensim) and applied neural language modeling techniques.

### 3. Generative AI and Large Language Models
**Question:**  
- What is your experience with generative AI and large language models? How have you leveraged fine-tuning to create domain-specific AI solutions?  
- How do you address challenges related to AI governance and ethical implementation in generative AI applications?  

**Talking Points:**  
- In-depth experience fine-tuning Meta’s LLaMA for email composition automation, demonstrating customization of large language models for business needs.  
- Created multi-agent systems capable of autonomous function execution using LLM orchestration with LangChain and CrewAI tools.  
- Passionate about AI governance and ethical AI development, integrating these principles in project design to ensure responsible innovation.

### 4. Cloud Computing & Data Engineering
**Question:**  
- What hands-on experience do you have with cloud computing platforms and relational databases? How do you optimize AI pipelines for real-time data processing?  

**Talking Points:**  
- Experience with cloud computing environments to support scalable AI pipelines, though specific cloud platforms can be elaborated during interview.  
- Implemented real-time web scraping and search integration to enhance NLP systems, showcasing ability to build data pipelines handling real-time conversational data and documents.  
- Worked on systems interfacing with Aadhaar databases demonstrating secure government interface and data handling.

### 5. Evaluation of ML Models and Performance Metrics
**Question:**  
- How do you evaluate the quality of your machine learning and NLP models? Which performance metrics do you prioritize and why?  

**Talking Points:**  
- Employs industry-standard evaluation metrics aligned with project goals, e.g., accuracy, precision/recall, F1-score for classification tasks, perplexity for language models.  
- Focus on performance balanced with scalability and real-world applicability, illustrated in projects like facial recognition system “Khoya Paya” for social good.  
- Understands importance of defining metrics in close consultation with stakeholders to ensure alignment with product platform requirements.

### 6. Technical Skills – Python and Tools Ecosystem
**Question:**  
- What Python tools and libraries are you most proficient with in data science and machine learning projects?  
- Describe your experience using libraries such as Pandas, NumPy, SciPy, scikit-learn, NLTK, and Gensim in actual projects.  

**Talking Points:**  
- Proficient in Python with extensive use of Pandas, NumPy, SciPy for data manipulation and numerical computing.  
- Used scikit-learn for developing classical ML models, NLTK and Gensim extensively for NLP tasks like text classification, parsing, and information extraction.  
- Also familiar with deep learning frameworks TensorFlow and PyTorch for CNNs and LLM fine-tuning tasks.

### 7. Leadership and Communication
**Question:**  
- How do you communicate complex technical results to non-technical stakeholders? Can you provide an example?  
- Describe an experience where you had to lead a team or influence cross-functional collaboration in AI projects.  

**Talking Points:**  
- Strong communication skills demonstrated by successful collaboration with non-technical founders and business teams, effectively translating AI capabilities into business terms.  
- Led multi-disciplinary teams in AI projects, clarifying roles and milestones (e.g., coordinating roles of Researcher, Writer, Editor agents in article generator).  
- Experience presenting project outcomes and technical recommendations clearly and concisely to diverse audiences.

### 8. Social Impact and Innovation
**Question:**  
- Can you share your experience developing AI solutions aimed at social good? What impact did they have?  

**Talking Points:**  
- Developed “Khoya Paya,” a facial recognition system linked with Aadhaar databases to aid government agencies in tracing missing persons, highlighting ethical AI application and social responsibility.  
- Project demonstrates application of AI in real-world societal challenges beyond commercial use cases.

### 9. Alignment with Rampp.ai and Role Expectations
**Question:**  
- Why are you interested in this Lead Data Scientist role at Rampp.ai and how do you see yourself contributing to the RADI Intelligence platform?  

**Talking Points:**  
- Enthusiastic about building innovative AI/ML products from scratch as described in the role summary, aligning with passion for product-oriented AI development.  
- Bringing strong technical expertise, leadership skills, and experience with contemporary NLP and ML techniques to contribute differentiated capabilities in the conversational AI domain.  
- Ready to leverage knowledge of generative AI and AI governance to help ensure platform excellence and ethical innovation.

---

## Additional Preparation Tips for Vansh

- Be ready to discuss technical details of your projects, focusing on challenges, solutions implemented, and outcomes.  
- Prepare examples demonstrating your ability to manage data pipelines and real-time data analysis.  
- Highlight ability to work with cloud infrastructure and database environments, even if experience is academic or project-based.  
- Rehearse explaining complex AI/ML concepts clearly and succinctly for non-technical stakeholders.  
- Show enthusiasm for continuous learning, leadership, and making impactful contributions in a dynamic startup environment.  

---

This comprehensive set of questions and talking points is designed to help you showcase your strengths, bridge your experience with the job requirements, and communicate your suitability confidently for the Lead Data Scientist role at Rampp.ai.

Good luck, Vansh! You are well prepared to make a strong impression.  
---